In [1]:
#default_exp query
%load_ext autoreload
%autoreload 2

# Queries

> Este módulo executa as queries sql / MongoDB necessárias para baixar os dados do STEL, RADCOM e MOSAICO

In [2]:
#export
import requests
import re
from decimal import *
from typing import *
from gazpacho import Soup
from rich.progress import track
from pathlib import Path
from unidecode import unidecode
import pandas as pd
import pandas_read_xml as pdx
import pyodbc
import collections
from fastcore.utils import listify
from fastcore.foundation import L
from fastcore.test import *
from anateldb.constants import *
from pyarrow import ArrowInvalid
from geopy.distance import geodesic

getcontext().prec = 5

BW = {'H': 0.001, 'K': 1, 'M': 1000, 'G': 1000000}
BW_pattern = re.compile("^(\d{1,3})([HKMG])(\d{0,2})$")
BW_MAP = {'230': '256K', '231': '256K', '205': '10K0', '800': '6M00', '801': '5M70', '248': '6M00', '247': '5M70', '167': '6M00'}


In [3]:
pasta = Path('c:/Users/rsilva/db')

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [4]:
#export
def optimize_floats(df: pd.DataFrame, exclude=None) -> pd.DataFrame:
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


def optimize_ints(df: pd.DataFrame, exclude=None) -> pd.DataFrame:
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


def optimize_objects(
    df: pd.DataFrame, datetime_features: List[str], exclude=None
) -> pd.DataFrame:
    for col in df.select_dtypes(include=["object"]).columns.tolist():
        if col not in datetime_features:
            if col in listify(exclude):
                continue
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                dtype = "category"
            else:
                dtype = "string"
            df[col] = df[col].astype(dtype)
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


def df_optimize(df: pd.DataFrame, datetime_features: List[str] = None, exclude=None):
    if datetime_features is None:
        datetime_features = []
    return optimize_floats(
        optimize_ints(optimize_objects(df, datetime_features, exclude), exclude),
        exclude,
    )

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [5]:
#export
def connect_db():
    """Conecta ao Banco ANATELBDRO01 e retorna o 'cursor' (iterador) do Banco pronto para fazer iterações"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=ANATELBDRO01;"
        "Database=SITARWEB;"
        "Trusted_Connection=yes;"
        "MultipleActiveResultSets=True;",
        timeout=TIMEOUT,
    )

In [6]:
#slow
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM, STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

## Funções auxiliares de Formatação
As funções a seguir são utilizadas para formatar diversos objetos intermediários utilizados nas funções de leitura e atualização da base de dados e não são chamadas diretamente. 

In [7]:
#exporti
def row2dict(row):
    """Receives a json row and return the dictionary from it"""
    return dict(row.items())


def dict2cols(df, reject=()):
    """Recebe um dataframe com dicionários nas células e extrai os dicionários como colunas
    Opcionalmente ignora e exclue as colunas em reject
    """
    for column in df.columns:
        if column in reject:
            df.drop(column, axis=1, inplace=True)
            continue
        if type(df[column].iloc[0]) == collections.OrderedDict:
            try:
                new_df = pd.DataFrame(df[column].apply(row2dict).tolist())
                df = pd.concat([df, new_df], axis=1)
                df.drop(column, axis=1, inplace=True)
            except AttributeError:
                continue
    return df


def parse_plano_basico(row, cols=COL_PB):
    """Receives a json row and filter the column in `cols`"""
    return {k: row[k] for k in cols}


def scrape_dataframe(id_list):
    df = pd.DataFrame()
    for id_ in track(id_list, description="Baixando informações complementares da Web"):
        html = requests.get(ESTACAO.format(id_))
        df = df.append(pd.read_html(Soup(html.text).find("table").html)[0])

    df.rename(
        columns={"NumFistel": "Fistel", "Num Serviço": "Num_Serviço"}, inplace=True
    )
    return df[
        [
            "Status",
            "Entidade",
            "Fistel",
            "Frequência",
            "Classe",
            "Num_Serviço",
            "Município",
            "UF",
        ]
    ]

In [8]:
#exporti
def input_coordenates(df, pasta):
    """Input the NA's in Coordinates with those of the cities"""
    municipios = Path(f"{pasta}/municípios.fth")
    if not municipios.exists():
        municipios = Path(f"{pasta}/municípios.xlsx")
        if not municipios.exists():
            raise FileNotFoundError(
                f"É necessario a tabela de municípios municípios.fth | municípios.xlsx na pasta {pasta}"
            )
        m = pd.read_excel(municipios, engine="openpyxl")
    else:
        m = pd.read_feather(municipios)
    m.loc[
        m.Município == "Sant'Ana do Livramento", "Município"
    ] = "Santana do Livramento"
    m["Município"] = m.Município.apply(unidecode).str.lower().str.replace("'", " ")
    m["UF"] = m.UF.str.lower()
    df["Coordenadas_do_Município"] = False
    df["Latitude"] = df.Latitude.str.replace(",", ".")
    df["Longitude"] = df.Longitude.str.replace(",", ".")
    df.loc[df["Município"] == "Poxoréo", "Município"] = "Poxoréu"
    df.loc[df["Município"] == "Couto de Magalhães", "Município"] = "Couto Magalhães"
    df["Município"] = df.Município.astype("string")
    criteria = (
        (df.Latitude == "")
        | (df.Latitude.isna())
        | (df.Longitude == "")
        | (df.Longitude.isna())
    ) & df.Município.isna()
    df = df[~criteria]
    for row in df[
        (
            (df.Latitude == "")
            | (df.Latitude.isna())
            | (df.Longitude == "")
            | (df.Longitude.isna())
        )
    ].itertuples():
        try:
            left = unidecode(row.Município).lower()
            m_coord = (
                m.loc[
                    (m.Município == left) & (m.UF == row.UF.lower()),
                    ["Latitude", "Longitude"],
                ]
                .values.flatten()
                .tolist()
            )
            df.loc[row.Index, "Latitude"] = m_coord[0]
            df.loc[row.Index, "Longitude"] = m_coord[1]
            df.loc[row.Index, "Coordenadas_do_Município"] = True
        except ValueError:
            print(left, row.UF, m_coord)
            continue
    return df


def clean_merge(pasta, df):
    df = df.copy()
    COLS = [c for c in df.columns if "_x" in c]
    for col in COLS:
        col_x = col
        col_y = col.split("_")[0] + "_y"
        if df[col_x].count() > df[col_y].count():
            a, b = col_x, col_y
        else:
            a, b = col_y, col_x

        df.loc[df[a].isna(), a] = df.loc[df[a].isna(), b]
        df.drop(b, axis=1, inplace=True)
        df.rename({a: a[:-2]}, axis=1, inplace=True)

    df.loc[df.Latitude_Transmissor == "", "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor == "", "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor == "", "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor == "", "Longitude_Estação"
    ]
    df.loc[df.Latitude_Transmissor.isna(), "Latitude_Transmissor"] = df.loc[
        df.Latitude_Transmissor.isna(), "Latitude_Estação"
    ]
    df.loc[df.Longitude_Transmissor.isna(), "Longitude_Transmissor"] = df.loc[
        df.Longitude_Transmissor.isna(), "Longitude_Estação"
    ]
    df.drop(["Latitude_Estação", "Longitude_Estação"], axis=1, inplace=True)
    df.rename(
        columns={
            "Latitude_Transmissor": "Latitude",
            "Longitude_Transmissor": "Longitude",
        },
        inplace=True,
    )

    df = input_coordenates(df, pasta)

    df["Frequência"] = df.Frequência.str.replace(",", ".")

    if freq_nans := df[df.Frequência.isna()].Id.tolist():
        complement_df = scrape_dataframe(freq_nans)
        df.loc[
            df.Frequência.isna(),
            [
                "Status",
                "Entidade",
                "Fistel",
                "Frequência",
                "Classe",
                "Num_Serviço",
                "Município",
                "UF",
            ],
        ] = complement_df.values

    for r in df[(df.Entidade.isna()) | (df.Entidade == "")].itertuples():
        df.loc[r.Index, "Entidade"] = ENTIDADES.get(r.Fistel, "")

    df.loc[df["Número_da_Estação"] == "", "Número_da_Estação"] = -1
    df["Latitude"] = df["Latitude"].astype("float")
    df["Longitude"] = df["Longitude"].astype("float")
    df["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Serviço == "OM", "Frequência"] = df.loc[
        df.Serviço == "OM", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df["Frequência"] = df.Frequência.astype("float")
    df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    df.loc[df["Num_Ato"] == "", "Num_Ato"] = -1
    df["Num_Ato"] = df.Num_Ato.astype("string")
    df["Num_Serviço"] = df.Num_Serviço.astype("category")
    return df_optimize(df, exclude=["Frequência"])

def parse_bw(bw):
    if match := re.match(BW_pattern, bw):
        multiplier = BW[match.group(2)]
        if mantissa := match.group(3):
            number = float(f'{match.group(1)}.{mantissa}')
        else:
            number = float(match.group(1))
        return multiplier * number
    return -1

## Funções de Leitura

In [9]:
#exporti
def read_estações(path):
    def extrair_ato(row):
        if not isinstance(row, str):
            row = listify(row)[::-1]
            for d in row:
                if not isinstance(d, dict):
                    continue
                if (d.get("@TipoDocumento") == "Ato") and (
                    d.get("@Razao") == "Autoriza o Uso de Radiofrequência"
                ):
                    return d["@NumDocumento"], d["@DataDOU"][:10]
            return "", ""
        return "", ""

    es = pdx.read_xml(path, ["estacao_rd"])
    dfs = []
    for i in range(es.shape[0]):
        df = pd.DataFrame(es["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(
        drop=True
    )
    docs = L(df.historico_documentos.apply(extrair_ato).tolist())
    df = df.loc[:, COL_ESTACOES]
    df["Num_Ato"] = docs.itemgot(0).map(str)
    df["Data_Ato"] = docs.itemgot(1).map(str)
    df.columns = NEW_ESTACOES
    df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    df["Data_Ato"] = df.Data_Ato.str.slice(0, 10)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    return df


def read_plano_basico(path):
    pb = pdx.read_xml(path, ["plano_basico"])
    dfs = []
    for i in range(pb.shape[0]):
        df = pd.DataFrame(pb["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.columns = NEW_PB
    df.sort_values(["Id", "Canal"], inplace=True)
    ENTIDADES.update(
        {r.Fistel: r.Entidade for r in df.itertuples() if str(r.Entidade) == "<NA>"}
    )
    df = df.groupby("Id", as_index=False).first()  # remove duplicated with NaNs
    df.dropna(subset=["Status"], inplace=True)
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(
        drop=True
    )
    return df

## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [10]:
#export
def update_radcom(pasta):
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(RADCOM, conn)
            df["Unidade"] = "MHz"
            df = df_optimize(df, exclude=["Frequência"])
            try:
                df.to_feather(f"{pasta}/radcom.fth")
            except ArrowInvalid:
                Path(f"{pasta}/radcom.fth").unlink()
                df.to_excel(f"{pasta}/radcom.xlsx", engine="openpyxl", index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_stel(pasta):
    """Atualiza a tabela local retornada pela query `STEL`"""
    with console.status(
        "[magenta]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="moon",
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(STEL, conn)
            df["Validade_RF"] = df.Validade_RF.astype("str").str.slice(0, 10)
            df["Num_Serviço"] = df.Num_Serviço.astype("category")
            df.loc[df.Unidade == "kHz", "Frequência"] = df.loc[
                df.Unidade == "kHz", "Frequência"
            ].apply(lambda x: Decimal(x) / Decimal(1000))
            df.loc[df.Unidade == "GHz", "Frequência"] = df.loc[
                df.Unidade == "GHz", "Frequência"
            ].apply(lambda x: Decimal(x) * Decimal(1000))
            df["Frequência"] = df.Frequência.astype("float")
            df.loc[df.Unidade == "kHz", "Unidade"] = "MHz"
            df = df_optimize(df, exclude=["Frequência"])
            try:
                df.to_feather(f"{pasta}/stel.fth")
            except ArrowInvalid:
                Path(f"{pasta}/stel.fth").unlink()
                df.to_excel(f"{pasta}/stel.xlsx", engine="openpyxl", index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )


def update_mosaico(pasta):
    """Atualiza a tabela local do Mosaico. É baixado e processado arquivos xml zipados da página pública do Spectrum E"""
    with console.status(
        "[blue]Baixando as Estações do Mosaico...", spinner="shark"
    ) as status:
        file = requests.get(ESTACOES)
        with open(f"{pasta}/estações.zip", "wb") as estações:
            estações.write(file.content)
    with console.status(
        "[blue]Baixando o Plano Básico das Estações...", spinner="weather"
    ) as status:
        file = requests.get(PLANO_BASICO)
        with open(f"{pasta}/Canais.zip", "wb") as plano_basico:
            plano_basico.write(file.content)
    console.print(":package: [blue]Consolidando as bases de dados...")
    estações = read_estações(f"{pasta}/estações.zip")
    plano_basico = read_plano_basico(f"{pasta}/Canais.zip")
    df = estações.merge(plano_basico, on="Id", how="left")
    df["Número_da_Estação"] = df["Número_da_Estação"].fillna(-1)
    df["Número_da_Estação"] = df["Número_da_Estação"].astype("int")
    df = clean_merge(pasta, df)
    try:
        df.reset_index(drop=True).to_feather(f"{pasta}/mosaico.fth")
    except ArrowInvalid:
        Path(f"{pasta}/mosaico.fth").unlink()
        with pd.ExcelWriter(f"{pasta}/mosaico.xlsx") as workbook:
            df.reset_index(drop=True).to_excel(
                workbook, sheet_name="Sheet1", engine="openpyxl", index=False
            )
    console.print("Kbô :vampire:")
    return df


def _merge_df(df1, df2, on, how="left"):
    """Merge two dataframes with the same columns and records"""
    df = pd.merge(df1, df2, on=on, how=how)
    x = df.Description_x.notna()
    y = df.Description_y.notna()
    df.loc[x & y, "Description"] = (
        df.loc[x & y, "Description_x"] + " | " + df.loc[x & y, "Description_y"]
    )
    df.loc[x & ~y, "Description"] = df.loc[x & ~y, "Description_x"]
    df.loc[~x & y, "Description"] = df.loc[~x & y, "Description_y"]
    df.loc[x & y, "Latitude"] = (
        df.loc[x & y, "Latitude_x"] + df.loc[x & y, "Latitude_y"]
    ) / 2
    df.loc[x & y, "Longitude"] = (
        df.loc[x & y, "Longitude_x"] + df.loc[x & y, "Longitude_y"]
    ) / 2
    df.loc[x & ~y, "Latitude"] = df.loc[x & ~y, "Latitude_x"]
    df.loc[x & ~y, "Longitude"] = df.loc[x & ~y, "Longitude_x"]
    df.loc[~x & y, "Latitude"] = df.loc[~x & y, "Latitude_y"]
    df.loc[~x & y, "Longitude"] = df.loc[~x & y, "Longitude_y"]
    if "Service_x" in df.columns and "Service_y" in df.columns:
        df.loc[x, "Service"] = df.loc[x, "Service_x"]
        df.loc[~x & y, "Service"] = df.loc[~x & y, "Service_y"]
    return df.loc[:, [c for c in df.columns if "_" not in c]]


def _merge_common(dfa, dfb, dfc):
    cols = ["Frequency", "Station"]
    a = dfa[dfa.Station != -1].reset_index(drop=True)
    b = dfb[dfb.Station != -1].reset_index(drop=True)
    c = dfc[dfc.Station != -1].reset_index(drop=True)
    common = _merge_df(a, b, cols, how="outer")
    common = _merge_df(common, c, cols, how="outer")
    common.drop_duplicates(inplace=True, keep="first")
    return common.reset_index(drop=True)


def _merge_new(dfa, dfb, dfc, dist=500):
    cols = ["Frequency"]
    a = (
        dfa[dfa.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    b = (
        dfb[dfb.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    c = (
        dfc[dfc.Station == -1]
        .drop(columns=["Service", "Station"])
        .reset_index(drop=True)
    )
    abc = (
        pd.merge(a, b, on=cols, how="outer")
        .merge(c, on=cols, how="outer")
        .reset_index(drop=True)
        .drop_duplicates(keep="first")
    )
    x = abc.Description_x.notna()
    y = abc.Description_y.notna()
    z = abc.Description.notna()
    new = pd.DataFrame(columns=["Frequency", "Latitude", "Longitude", "Description"])

    for c, n in zip([(x & ~y & ~z), (~x & y & ~z), (~x & ~y & z)], ["_x", "_y", ""]):
        temp = abc.loc[
            c, ["Frequency", f"Latitude{n}", f"Longitude{n}", f"Description{n}"]
        ]
        temp.columns = ["Frequency", "Latitude", "Longitude", "Description"]
        new = pd.concat([new, temp], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for c, (left, right) in zip(
        [(x & y & ~z), (x & ~y & z), (~x & y & z)],
        [("_x", "_y"), ("_x", ""), ("_y", "")],
    ):
        for row in abc[c].itertuples():
            d = geodesic(
                (getattr(row, f"Latitude{left}"), getattr(row, f"Longitude{left}")),
                (getattr(row, f"Latitude{right}"), getattr(row, f"Longitude{right}")),
            ).m
            if d < dist:
                f = row.Frequency
                lat = (
                    getattr(row, f"Latitude{left}") + getattr(row, f"Latitude{right}")
                ) / 2
                lon = (
                    getattr(row, f"Longitude{left}") + getattr(row, f"Longitude{right}")
                ) / 2
                d = (
                    getattr(row, f"Description{left}")
                    + " | "
                    + getattr(row, f"Description{right}")
                )
                new = pd.concat(
                    [
                        new,
                        pd.DataFrame(
                            [[f, lat, lon, d]],
                            columns=[
                                "Frequency",
                                "Latitude",
                                "Longitude",
                                "Description",
                            ],
                        ),
                    ],
                    ignore_index=True,
                )
            else:
                l = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{left}",
                            f"Longitude{left}",
                            f"Description{left}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                l.columns = new.columns
                r = (
                    abc.loc[
                        row.Index,
                        [
                            "Frequency",
                            f"Latitude{right}",
                            f"Longitude{right}",
                            f"Description{right}",
                        ],
                    ]
                    .to_frame()
                    .T
                )
                r.columns = new.columns
                new = pd.concat([new, l, r], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)

    for row in abc[x & y & z].itertuples():
        d1 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
        ).m
        d2 = geodesic(
            (getattr(row, "Latitude_x"), getattr(row, "Longitude_x")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        d3 = geodesic(
            (getattr(row, "Latitude_y"), getattr(row, "Longitude_y")),
            (getattr(row, "Latitude"), getattr(row, "Longitude")),
        ).m
        if d1 < dist and d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y + row.Latitude) / 3
            lon = (row.Longitude_x + row.Longitude_y + row.Longitude) / 3
            d = " | ".join([row.Description_x, row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d2 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude_y) / 2
            lon = (row.Longitude_x + row.Longitude_y) / 2
            d = " | ".join([row.Description_x, row.Description_y])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d1 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_x + row.Latitude) / 2
            lon = (row.Longitude_x + row.Longitude) / 2
            d = " | ".join([row.Description_x, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        elif d2 < dist and d3 < dist:
            f = row.Frequency
            lat = (row.Latitude_y + row.Latitude) / 2
            lon = (row.Longitude_y + row.Longitude) / 2
            d = " | ".join([row.Description_y, row.Description])
            new = pd.concat(
                [
                    new,
                    pd.DataFrame(
                        [[f, lat, lon, d]],
                        columns=["Frequency", "Latitude", "Longitude", "Description"],
                    ),
                ],
                ignore_index=True,
            )
        else:
            l = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_x", "Longitude_x", "Description_x"],
                ]
                .to_frame()
                .T
            )
            l.columns = new.columns
            r = (
                abc.loc[
                    row.Index,
                    ["Frequency", "Latitude_y", "Longitude_y", "Description_y"],
                ]
                .to_frame()
                .T
            )
            r.columns = new.columns
            s = (
                abc.loc[
                    row.Index, ["Frequency", "Latitude", "Longitude", "Description"]
                ]
                .to_frame()
                .T
            )
            s.columns = new.columns
            new = pd.concat([new, l, r, s], ignore_index=True)

    new = new.drop_duplicates(keep="first").reset_index(drop=True)
    new["Service"] = -1
    new["Station"] = -1
    return new


def read_aero(pasta, up_icao=False, up_pmec=False, up_geo=False):
    icao = read_icao(pasta, up_icao)
    pmec = read_pmec(pasta, up_pmec)
    geo = read_geo(pasta, up_geo)
    icao["Description"] = icao.Description.astype("string")
    pmec["Description"] = pmec.Description.astype("string")
    geo["Description"] = geo.Description.astype("string")
    common = _merge_common(icao, pmec, geo)
    new = _merge_new(icao, pmec, geo)
    return common, new


def merge_aero(df, common, new):
    """Mescla a base da Anatel com as tabelas retiradas da Aeronáutica"""
    common = common.loc[:, ["Frequency", "Description", "Service", "Station"]]
    df["Description"] = df.Description.astype("string")
    df = pd.merge(df, common, on=["Frequency", "Service", "Station"], how="left")
    df.loc[df.Description_y.notna(), "Description_x"] = (
        df.loc[df.Description_y.notna(), "Description_x"]
        + " | "
        + df.loc[df.Description_y.notna(), "Description_y"]
    )
    df.drop(columns=["Description_y"], inplace=True)
    df.rename(columns={"Description_x": "Description"}, inplace=True)
    new['Class'] = '-1'
    new['BW'] = '-1'
    return (
        pd.concat([df, new], ignore_index=True)
        .sort_values("Frequency")
        .reset_index(drop=True)
    )

def update_base(pasta, up_stel=False, up_radcom=False, up_mosaico=False, up_icao=False):
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na pasta `pasta`"""
    stel = read_stel(pasta, up_stel).loc[:, TELECOM]
    radcom = read_radcom(pasta, up_radcom)
    mosaico = read_mosaico(pasta, up_mosaico)
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = ""
    radcom["Largura_Emissão"] = BW_MAP['231']
    radcom["Classe"] = f'{radcom.Fase.str.strip()}-{radcom.Situação.str.strip()}'
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Num_Ato"] = "-1"
    radcom["Data_Ato"] = ""
    radcom["Validade_RF"] = ""
    radcom["Fonte"] = "SRD"
    radcom = df_optimize(radcom, exclude=["Frequência"])
    stel["Status"] = "L"
    stel["Num_Ato"] = "-1"
    stel["Data_Ato"] = ""
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    stel = df_optimize(stel, exclude=["Frequência"])
    mosaico["Fonte"] = "MOS"
    mosaico["Classe_Emissão"] = ""
    mosaico["Largura_Emissão"] = mosaico.Num_Serviço.map(BW_MAP)
    mosaico = mosaico.loc[:, RADIODIFUSAO]
    mosaico = df_optimize(mosaico, exclude=["Frequência"])
    rd = (
        pd.concat([mosaico, radcom, stel])
        .sort_values("Frequência")
        .reset_index(drop=True)
    )
    rd["Num_Serviço"] = rd.Num_Serviço.astype("int")
    rd = df_optimize(rd, exclude=["Frequência"])
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)
    rd['BW(kHz)'] = rd.Largura_Emissão.apply(parse_bw)
    console.print(":trophy: [green]Base Consolidada. Salvando os arquivos...")
    try:
        rd.to_feather(f"{pasta}/base.fth")
    except ArrowInvalid:
        Path(f"{pasta}/base.fth").unlink()
        with pd.ExcelWriter(f"{pasta}/base.xlsx") as workbook:
            rd.to_excel(workbook, sheet_name="Sheet1", engine="openpyxl", index=False)
    return rd

## Funções de Leitura das diversas bases
A presente biblioteca utiliza três bases de dados: 
* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão comunitária
* `MOSAICO` - Demais serviços de Radiodifusão e paulatinamente também adicionado serviços privados

As funções a seguir são para leitura dos arquivos locais baixados e processados dessas bases, opcionalmente esses arquivos podem ser atualizados antes de serem lidos passando o argumento `update = True`

In [11]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_stel(pasta)
    file = Path(f"{pasta}/stel.fth")
    try:
        stel = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/stel.xlsx")
        try:
            stel = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_stel(pasta, True)

    return stel


def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    if update:
        update_radcom(pasta)
    file = Path(f"{pasta}/radcom.fth")
    try:
        radcom = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/radcom.xlsx")
        try:
            radcom = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            read_radcom(pasta, True)
    return radcom


def read_mosaico(pasta, update=False):
    """Lê o banco de dados salvo localmente do MOSAICO. Opcionalmente o atualiza antes da leitura baixando os diversos arquivos disponíveis na interface web pública"""
    if update:
        update_mosaico(pasta)
    file = Path(f"{pasta}/mosaico.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/mosaico.xlsx")
        try:
            df = pd.read_excel(file)
        except FileNotFoundError:
            return read_mosaico(pasta, update=True)
    return df


def read_icao(pasta, update=False):
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    if update:
        # TODO: atualizar a base de dados do Frequency Finder e Canalização VOR/ILS/DME
        # update_icao(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/IcaoDB.fth")
    try:
        icao = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/IcaoDB.xlsx")
        try:
            icao = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return df_optimize(icao, exclude=["Frequency"])


def read_pmec(pasta, update=False):
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME
        # update_pmec(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/PmecDB.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/PmecDB.xlsx")
        try:
            df = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return df_optimize(df, exclude=["Frequency"])


def read_geo(pasta, update=False):
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do GEOAISWEB
        # update_geo(pasta)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    file = Path(f"{pasta}/GeoAiswebDB.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/GeoAiswebDB.xlsx")
        try:
            df = pd.read_excel(file, sheet_name="DataBase", engine="openpyxl")
        except FileNotFoundError:
            return read_icao(pasta, update=True)
    return df_optimize(df, exclude=["Frequency"])


def read_base(pasta, up_stel=False, up_radcom=False, up_mosaico=False, up_icao=False):
    """Wrapper que combina a chamada das três funções de leitura do banco e opcionalmente é possível atualizá-las antes da leitura"""
    if any([up_stel, up_radcom, up_mosaico, up_icao]):
        update_base(pasta, up_stel, up_radcom, up_mosaico, up_icao)
    file = Path(f"{pasta}/base.fth")
    try:
        df = pd.read_feather(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{pasta}/base.xlsx")
        try:
            df = pd.read_excel(file, engine="openpyxl")
        except FileNotFoundError:
            df = update_base(pasta, True, True, True)
    return df_optimize(df, exclude=["Frequência"])

In [12]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
